# FEniCSx Tutorial

In this tutorial, we will be looking at the FEniCSx computing platform and seeing how it assists with FEA analysis. As we have seen in class, our starting point was going through simple elastic solvers and Newtonian methods. We will similarily begin this tutorial with an implementation of Linear Elasticity in 2 dimensions, or simulating a downward force on an elastic beam. Before getting into the problem, please ensure that FEniCSx and DolfinX are installed on your computer. For assistance with installation, please refer to the `readme.md` for tips on installing the platform, creating a conda environment, activating it (also in your terminal/visual studio session), and running the Jupyter Notebook.

## Library Imports

To ensure that FEniCSx runs properly, please run each Jupyter notebook cell in order. If you run any cells ahead of others, the code might error out. First and foremost, we call our imports, the main ones being Numpy, DolfinX, UFL. 

Here:

- Numpy:        Standard import which is for computations and arrays
- DolfinX:      Core FEniCSx library for FEA
- UFL:          Domain-specific language usedfor FEA


In [ ]:
import numpy as np
from mpi4py import MPI
from dolfinx import mesh, fem, plot
import ufl
import pyvista
from dolfinx.fem import petsc
from dolfinx.plot import vtk_mesh

## Simple Bridge Problem Setup

Next we need to initialize our problem variables. As discussed before, we are setting up a simple bridge which is fixed on both ends, and simulating a downward *point* force right in the middle of the beam. Please note that in this example, we are not applying uniform force and instead are testing how a single point heavy object might affect the bridge, like a semi carrying gravel. 

The steps for this problem will look like this:
1) Define mesh
2) Create the domain space (label the edges)
3) Define material properties

In [ ]:
# Create 2D rectangular mesh (bridge slab)
L, H = 4.0, 0.5
nx, ny = 80, 10 # num of elements which are hopefully not above threshold test
domain = mesh.create_rectangle(MPI.COMM_WORLD,
                               [np.array([0.0, 0.0]), np.array([L, H])],
                               [nx, ny],
                               cell_type=mesh.CellType.triangle)

# function space for displacement
V = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim, )))

# Material properties supposedly resembling steel
E = 2e11  # Young’s modulus
nu = 0.3  # Poisson
mu = E / (2 * (1 + nu))
lmbda = E * nu / ((1 + nu) * (1 - 2 * nu))

## Stress & Strain Simplicity

In this next section we are defining stress and strain functions which are used for simplicity's sake. There are built in gradient functions within ufl which can calculate this, but to make our lives easier and our code more readable (and follow the math that we went over), we explicitly define stress and strain.

In [ ]:
# Defining strain and stress 
def epsilon(u):
    return ufl.sym(ufl.grad(u))

def sigma(u):
    return lmbda * ufl.tr(epsilon(u)) * ufl.Identity(len(u)) + 2 * mu * epsilon(u)

## Boundary Conditions

Simple boundary condition applications where we define where the left and right boundaries of the beam are. Left boundary conditions are at 0,0. Right boundary conditions are the length of the beam.

In [ ]:
def left(x): return np.isclose(x[0], 0.0)
def right(x): return np.isclose(x[0], L)
fixed_dofs = fem.locate_dofs_geometrical(V, lambda x: left(x) | right(x)) # this line locates the left and right edges of our rectangle
bc = fem.dirichletbc(np.array([0.0, 0.0], dtype=np.float64), fixed_dofs, V)

## UFL Commands

The following 3 commands are now called from UFL which are essential in our finite element analysis. We calculate the displacement by calling TrialFunction and passing our function space vector. We get the weak form by calling the TestFunction. Finally, we get the integral of the inner product that we learned about last couple of lectures. These commands are essential steps in our FEA pipeline.

In [ ]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V) 
a = ufl.inner(sigma(u), epsilon(v)) * ufl.dx 

## Force Application

Next we define a function called CenterTopLoad for more ease of use. This function will simply take the length of the beam and half it for the X coordinate, then use the height as the Y coordinate. We apply this load to the negative Y direction of the mesh by calling the meshtags function and passing the domain, topology (negative y), boundary properties, and load.  

In [ ]:
# Applying load on the top edge of the bridge/beam
class CenterTopLoad:
    def __init__(self, width=0.1):
        self.width = width
    def __call__(self, x):
        # Load around x = L/2 and top y = H
        return np.logical_and(np.isclose(x[1], H),
                              np.abs(x[0] - L/2) < self.width)

# Load is applied in the negative y-direction
boundary_facets = mesh.locate_entities_boundary(
    domain, domain.topology.dim - 1, CenterTopLoad(width=0.1)
)

# Create a meshtag for the boundary
load_markers = mesh.meshtags(domain, domain.topology.dim - 1, boundary_facets, 1)


## Apply and Solve

Finally, we create a constant donward force, and pass it to the solver.

In [ ]:
# force downward
f = fem.Constant(domain, np.array([0.0, 5e9], dtype=np.float64))  # N/m^2
ds = ufl.Measure("ds", domain=domain, subdomain_data=load_markers)
L_form = ufl.dot(f, v) * ds(1)

# Solve
problem = petsc.LinearProblem(a, L_form, bcs=[bc])
u_h = problem.solve()

## Visualization

We are going to visualize this in two ways. The first by creating a grid of the mesh itself, and then plotting the mesh. Note that we are not using the standard pyplot functions as we are not using that library at all. FEniCSx works incredibly well when using the pyvista library, as it specializes in vector and array plotting, handling 3 dimensional simulation with ease.

In [ ]:
pyvista.start_xvfb()
plotter = pyvista.Plotter()
plotter.open_gif("deformation.gif", fps=3)

topology, cell_types, geometry = vtk_mesh(V)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

dim = 2  # This is a 2D problem
n_verts = geometry.shape[0]  # Should be 891
disp = u_h.x.array.reshape(n_verts, dim)

# Pad to 3D
values = np.zeros((n_verts, 3))
values[:, :dim] = disp

# Assign to grid
grid["u"] = values
grid.set_active_vectors("u")

for factor in np.linspace(0, 1.0, 20):  # 20 frames
    warped = grid.warp_by_vector("u", factor=factor)
    warped.set_active_vectors("u")
    plotter.clear()  # Clear previous mesh
    disp_mag = np.linalg.norm(values, axis=1)
    warped["disp_mag"] = disp_mag
    plotter.add_mesh(warped, scalars="disp_mag", cmap="viridis", show_edges=True)
    plotter.write_frame()

plotter.close()


## Results

We have a saved `deformation.gif` file saved in the same directory. This shows us the deformation that occurs on our bridge when we apply a force in the NEGATIVE Y direction. It is important to note that this bridge is in 2 Dimensions. If we wanted to work in 3D, we would need to define our elements, mesh, and the rest of the problem in all 3 dimensions, including the Z axis. 

### Challenge

1) Modify this jupyter notebook and create a downward force in the Z direction in 3 dimensions.
2) Save the gif and see the effects.